In [1114]:
import pandas as pd

In [1115]:
data_imdb = pd.read_csv("input_datasets/fusion_without_release&nbrEntry.csv")
data_target = pd.read_csv("input_datasets/allocine.csv")




In [1116]:
display(data_target.info())
display(data_imdb.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   actors          1680 non-null   object 
 1   director        1721 non-null   object 
 2   entrees_fr      1726 non-null   int64  
 3   entrees_usa     0 non-null      float64
 4   image_urls      1726 non-null   object 
 5   nationalite     1724 non-null   object 
 6   semaine_fr      1726 non-null   object 
 7   semaine_usa     0 non-null      float64
 8   studio          1647 non-null   object 
 9   timing          1726 non-null   int64  
 10  title           1726 non-null   object 
 11  titre_original  1699 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 161.9+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 0 to 1696
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   titre              1697 non-null   object
 1   score              1697 non-null   object
 2   nbre_vote          1697 non-null   object
 3   genres             1697 non-null   object
 4   langue             1697 non-null   object
 5   pays               1697 non-null   object
 6   pegi               1697 non-null   object
 7   duree              1697 non-null   object
 8   annee              1697 non-null   object
 9   popularite_score   1697 non-null   object
 10  director           1697 non-null   object
 11  scenaristes        1697 non-null   object
 12  casting_principal  1697 non-null   object
 13  casting_complet    1697 non-null   object
 14  budget             1697 non-null   object
dtypes: object(15)
memory usage: 199.0+ KB


None

Fusion de deux datasets: 

In [1117]:
import pandas as pd

# Renommer la colonne 'titre' du dataset 'data_imdb' pour correspondre à la colonne 'film_title' du dataset 'data_target'
# data_imdb.rename(columns={'titre': 'title'}, inplace=True)

data_imdb['titre'] = data_imdb['titre'].str.lower()
data_target['title'] = data_target['title'].str.lower()

# Fusionner les datasets en utilisant le titre du film comme clé de fusion
data = pd.merge(data_imdb, data_target, left_on='titre', right_on='title')

# Afficher le nouveau dataset fusionné
# display(data)


gestion probleme pegi et durée

In [1118]:
# data.loc[data['duree'] == 0, 'duree'] = data['pegi']
# if (data['duree'].equals(0)):
#     data['duree'] = data['pegi']
# print(data['duree'])
data.loc[data['duree'] == "0", 'duree'] = data['pegi']

# # Initialiser le compteur à zéro
# nb_duree_zero = 0

# # Parcourir chaque valeur de la colonne 'duree'
# for duree in data['duree']:
#     if duree == "0":
#         nb_duree_zero += 1

# print("Nombre de fois où la durée est égale à 0 :", nb_duree_zero)


Gestion de Pegi :

In [1119]:
data.loc[data['pegi'].str.match(r'^\d+'), 'pegi'] = None

# count_none = (data['pegi'] == "None").sum()
# print("Nombre de valeurs 'None' dans la colonne 'pegi' :", count_none)

Gestion durée:

In [1120]:
def convert_timing(timing):
    try:
        if 'h' in timing and 'min' in timing:
            timing_cleaned = timing.replace('min', '') 
            hours, minutes = map(int, timing_cleaned.split('h '))
            total_minutes = hours * 60 + minutes
        elif 'h' in timing:
            hours = int(timing.replace('h', ''))
            total_minutes = hours * 60
        elif 'min' in timing:
            total_minutes = int(timing.replace('min', ''))
        else:
            total_minutes = 0
        return total_minutes
    except:
        return None


data['duree']= data['duree'].apply(convert_timing)

gestion du nbbr_vote

In [1121]:
# for i, vote in enumerate(data['nbre_vote']):
    # if ',' in vote:
    #     data.at[i, 'nbre_vote'] = vote.replace(',', '')
    #     data.at[i, 'nbre_vote'] = data.at[i, 'nbre_vote'].replace(' k', '00')
    # else:
    #     data.at[i, 'nbre_vote'] = vote.replace(' k', '000')
def convert_vote(vote):
    try:
        if ',' in vote :
            vote_clean0 = vote.replace(',', '') 
            vote_clean = vote_clean0.replace("k", '00') 
        else:
            vote_clean = vote.replace("k", '000') 
        
        # vote_clean =  int(float(vote_clean))
        return vote_clean
    except:
        return None



data['nbre_vote']= data['nbre_vote'].apply(convert_vote)

In [1122]:
display(data)

,titre,score,nbre_vote,genres,langue,pays,pegi,duree,annee,popularite_score,...,entrees_fr,entrees_usa,image_urls,nationalite,semaine_fr,semaine_usa,studio,timing,title,titre_original
0,heureux gagnants,"6,3",190,['Comédie'],['Français'],['France'],None,103,2024,['No Popularity Score'],...,136628,NaN,https://fr.web.img6.acsta.net/c_310_420/pictur...,france,13/03/2024 au 20/03/2024,NaN,warner bros. france,0,heureux gagnants,pas de titre original
1,14 jours pour aller mieux,"5,6",50,['Comédie'],['Français'],['France'],None,90,2024,['No Popularity Score'],...,126678,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,france,06/03/2024 au 13/03/2024,NaN,wild bunch distribution,0,14 jours pour aller mieux,pas de titre original
2,la vie de ma mère,"6,9",79,"['Comédie', 'Drame']",['Français'],['France'],None,105,2023,['No Popularity Score'],...,58613,NaN,https://fr.web.img4.acsta.net/c_310_420/pictur...,france,06/03/2024 au 13/03/2024,NaN,kmbo,0,la vie de ma mère,pas de titre original
3,la zone d'intérêt,"7,5",73 000,"['Drame', 'Historique', 'Guerre']","['Allemand', 'Polonais', 'Yiddish']","['États-Unis', 'Royaume-Uni', 'Pologne']",Tous publics,105,2023,28,...,239108,NaN,https://fr.web.img6.acsta.net/c_310_420/pictur...,u.s.a.,31 janvier au 7 février 2024,NaN,bac films,0,la zone d'intérêt,the zone of interest
4,il reste encore demain,"7,7",64 00,"['Comédie', 'Drame']",['Italien'],['Italie'],Tous publics,118,2023,552,...,145340,NaN,https://fr.web.img3.acsta.net/c_310_420/pictur...,italie,13/03/2024 au 20/03/2024,NaN,universal pictures international france,0,il reste encore demain,c'è ancora domani
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336,ordinary angels,"7,5",35 00,['Drame'],['Anglais'],['États-Unis'],PG,118,2024,113,...,6500000,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,u.s.a.,23/02/2024 au 26/02/2024,NaN,NaN,0,ordinary angels,pas de titre original
1337,out of darkness,"5,6",34 00,"['Horreur', 'Thriller']",['No Language'],['Royaume-Uni'],R,88,2022,634,...,0,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,grande-bretagne,09/02/2024 au 12/02/2024,NaN,NaN,0,out of darkness,pas de titre original
1338,i.s.s.,"5,2",65 00,"['Science-fiction', 'Thriller']","['Anglais', 'Russe']",['États-Unis'],R,95,2023,1 502,...,3064583,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,u.s.a.,19/01/2024 au 22/01/2024,NaN,1 nomination,0,i.s.s.,pas de titre original
1339,the american society of magical negroes,"2,7",66 00,"['Comédie', 'Fantastique']",['Anglais'],"['États-Unis', 'Danemark']",PG-13,104,2024,45,...,1250000,NaN,https://fr.web.img6.acsta.net/c_310_420/pictur...,u.s.a.,15/03/2024 au 18/03/2024,NaN,NaN,0,the american society of magical negroes,pas de titre original


formatage date:

In [1123]:
mois_numeriques = {
    "janvier": "01",
    "février": "02",
    "mars": "03",
    "avril": "04",
    "mai": "05",
    "juin": "06",
    "juillet": "07",
    "août": "08",
    "septembre": "09",
    "octobre": "10",
    "novembre": "11",
    "décembre": "12"
}

# Fonction pour convertir le format de date pour allocine
def format_date_allocine(date_string):

    for m in mois_numeriques.keys():
        if m in date_string:
            splits = date_string.split()
            day = str(splits[0])
            month = str(mois_numeriques[splits[1]])
            year = str(splits[-1])
            date_format = day + "/" + month + "/" + year
            return date_format

    date_format = str(date_string.split(' au ')[0])
    return date_format

data['semaine_fr'] = data['semaine_fr'].apply(format_date_allocine)


In [1124]:
cols_drop = ["timing", "semaine_usa", "image_urls", "entrees_usa"]

data = data.drop(cols_drop, axis=1)

#par manque de données:
cols_drop2 = ["budget"]
data = data.drop(cols_drop2, axis=1)

#par manque de données:
data_leaking = ["score", "nbre_vote", "popularite_score"]
data = data.drop(data_leaking, axis=1)


display(data)

,titre,genres,langue,pays,pegi,duree,annee,director_x,scenaristes,casting_principal,casting_complet,actors,director_y,entrees_fr,nationalite,semaine_fr,studio,title,titre_original
0,heureux gagnants,['Comédie'],['Français'],['France'],None,103,2024,Romain Choay,Romain Choay,"['Romain Choay', 'Maxime Govare', 'Scénario']","['Fabrice Eboué', 'Audrey Lamy', 'Anouk Grinbe...","Audrey Lamy,Fabrice Eboué,Anouk Grinberg","Maxime Govare,Romain Choay,Maxime Govare,Romai...",136628,france,13/03/2024,warner bros. france,heureux gagnants,pas de titre original
1,14 jours pour aller mieux,['Comédie'],['Français'],['France'],None,90,2024,Edouard Pluvieux,Edouard Pluvieux,"['Edouard Pluvieux', 'Scénario', 'Benjamin Dem...","['Maxime Gasteuil', 'Zabou Breitman', 'Romain ...","Maxime Gasteuil,Zabou Breitman,Romain Lancry","Edouard Pluvieux,Lionel Dutemple,Edouard Pluvieux",126678,france,06/03/2024,wild bunch distribution,14 jours pour aller mieux,pas de titre original
2,la vie de ma mère,"['Comédie', 'Drame']",['Français'],['France'],None,105,2023,Julien Carpentier,Julien Carpentier,"['Julien Carpentier', 'Scénario', 'Julien Carp...","['Agnès Jaoui', 'William Lebghil', 'Salif Ciss...","Agnès Jaoui,William Lebghil,Salif Cissé","Julien Carpentier,Julien Carpentier,Benjamin G...",58613,france,06/03/2024,kmbo,la vie de ma mère,pas de titre original
3,la zone d'intérêt,"['Drame', 'Historique', 'Guerre']","['Allemand', 'Polonais', 'Yiddish']","['États-Unis', 'Royaume-Uni', 'Pologne']",Tous publics,105,2023,Jonathan Glazer,Jonathan Glazer,"['Jonathan Glazer', 'Jonathan Glazer', 'Martin...","['Christian Friedel', 'Sandra Hüller', 'Johann...","Christian Friedel,Sandra Hüller,Johann Karthaus","Jonathan Glazer,Jonathan Glazer",239108,u.s.a.,31/01/2024,bac films,la zone d'intérêt,the zone of interest
4,il reste encore demain,"['Comédie', 'Drame']",['Italien'],['Italie'],Tous publics,118,2023,Paola Cortellesi,Paola Cortellesi,"['Paola Cortellesi', 'Furio Andreotti', 'Giuli...","['Paola Cortellesi', 'Valerio Mastandrea', 'Ro...","Paola Cortellesi,Valerio Mastandrea,Romana Mag...","Paola Cortellesi,Paola Cortellesi,Furio Andreotti",145340,italie,13/03/2024,universal pictures international france,il reste encore demain,c'è ancora domani
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336,ordinary angels,['Drame'],['Anglais'],['États-Unis'],PG,118,2024,Jon Gunn,Jon Gunn,"['Jon Gunn', 'Meg Tilly', 'Kelly Fremon Craig']","['Hilary Swank', 'Alan Ritchson', 'Emily Mitch...","Hilary Swank,Alan Ritchson,Amy Acker","Jon Gunn,Kelly Fremon Craig",6500000,u.s.a.,23/02/2024,NaN,ordinary angels,pas de titre original
1337,out of darkness,"['Horreur', 'Thriller']",['No Language'],['Royaume-Uni'],R,88,2022,Andrew Cumming,Andrew Cumming,"['Andrew Cumming', 'Scénario', 'Ruth Greenberg']","['Iola Evans', 'Arno Lüning', 'Rosebud Melarke...","Chuku Modu,Kit Young,Safia Oakley-Green","Andrew Cumming,Ruth Greenberg",0,grande-bretagne,09/02/2024,NaN,out of darkness,pas de titre original
1338,i.s.s.,"['Science-fiction', 'Thriller']","['Anglais', 'Russe']",['États-Unis'],R,95,2023,Gabriela Cowperthwaite,Gabriela Cowperthwaite,"['Gabriela Cowperthwaite', 'Nick Shafir', 'Cas...","['Ariana DeBose', 'Chris Messina', 'John Galla...","Ariana DeBose,Chris Messina,Pilou Asbæk","Gabriela Cowperthwaite,Nick Shafir",3064583,u.s.a.,19/01/2024,1 nomination,i.s.s.,pas de titre original
1339,the american society of magical negroes,"['Comédie', 'Fantastique']",['Anglais'],"['États-Unis', 'Danemark']",PG-13,104,2024,Kobi Libii,Kobi Libii,"['Kobi Libii', 'Kobi Libii', 'Casting principal']","['Justice Smith', 'Zachary Barton', 'Anthony C...","Justice Smith,David Alan Grier,An-Li Bogan","Kobi Libii,Kobi Libii",1250000,u.s.a.,15/03/2024,NaN,the american society of magical negroes,pas de titre original


formatage durée :

In [1125]:
import pandas as pd

# Charger le dataset
data_imdb = pd.read_csv("input_datasets/imdbmovies.csv")

# Fonction pour vérifier si la durée est dans le format spécifié
def is_valid_timing(timing):
    return 'h' in timing and 'm' in timing

# Afficher les durées qui ne sont pas dans le format spécifié
invalid_timings = data_imdb[~data_imdb['duree'].apply(is_valid_timing)]
print("Durées qui ne sont pas dans le format spécifié 'xh ym':")
print(invalid_timings[['duree']])


Durées qui ne sont pas dans le format spécifié 'xh ym':
     duree
0        0
5        0
17       0
18       0
24       0
...    ...
2465    2h
2466     0
2470     0
2478    2h
2493    2h

[221 rows x 1 columns]


In [1126]:
def convert_timing(timing):
    try:
        if 'h' in timing and 'm' in timing:
            timing_cleaned = timing.replace('m', '')  # Supprimer 'm' de la chaîne
            hours, minutes = map(int, timing_cleaned.split('h '))
            total_minutes = hours * 60 + minutes
        elif 'h' in timing:
            hours = int(timing.replace('h', ''))
            total_minutes = hours * 60
        elif 'm' in timing:
            total_minutes = int(timing.replace('m', ''))
        else:
            total_minutes = 0
        return total_minutes
    except:
        return None


data_imdb['duree']= data_imdb['duree'].apply(convert_timing)


formatage date :


In [1127]:
mois_numeriques = {
    "janvier": "01",
    "février": "02",
    "mars": "03",
    "avril": "04",
    "mai": "05",
    "juin": "06",
    "juillet": "07",
    "août": "08",
    "septembre": "09",
    "octobre": "10",
    "novembre": "11",
    "décembre": "12"
}

# Fonction pour convertir le format de date pour allocine
def format_date_allocine(date_string):

    for m in mois_numeriques.keys():
        if m in date_string:
            splits = date_string.split()
            day = str(splits[0])
            month = str(mois_numeriques[splits[1]])
            year = str(splits[-1])
            date_format = day + "/" + month + "/" + year
            return date_format

    date_format = str(date_string.split(' au ')[0])
    return date_format

# Fonction pour convertir le format de date pour imdb
def convert_date_format(date_str):
    try:
        return pd.to_datetime(date_str).strftime('%d/%m/%Y')
    except:
        return None

# Convertir les dates du csv d'allocine
data_allocine['semaine_fr'] = data_allocine['semaine_fr'].apply(format_date_allocine)

# Appliquer la fonction pour convertir le format de date
data_imdb['date_sortie_y'] = data_imdb['date_sortie_y'].apply(convert_date_format)

# Supprime les données avec no date found / None
data_imdb = data_imdb[(data_imdb["date_sortie_y"] != 'No date found') & (data_imdb["date_sortie_y"].notna())]

# Afficher les premières lignes pour vérification
display(data_imdb["date_sortie_y"].info())

# Supprimer les données avec dates antérieurs à 2000

<class 'pandas.core.series.Series'>
Index: 2245 entries, 1 to 2499
Series name: date_sortie_y
Non-Null Count  Dtype 
--------------  ----- 
2245 non-null   object
dtypes: object(1)
memory usage: 35.1+ KB


None

Gestion du Budget:

In [1128]:
data_imdb['budget'] = data_imdb['budget'].str.replace(' (estimated)', '')

data_imdb['budget'] = data_imdb['budget'].str.replace(',', '')
data_imdb['budget'] = data_imdb['budget'].str.replace(' ', '')

# Supprimer les lignes avec 'No budger' dans la colonne 'budget'
data_imdb = data_imdb[data_imdb['budget'] != 'No budget']

# Réinitialiser l'index
data_imdb.reset_index(drop=True, inplace=True)

# Afficher les premières lignes pour vérification
# display(data_imdb)


In [1129]:
import re

taux_de_conversion = {
    '$': 1.0,
    '£': 1.38,  # Livre sterling (taux de change actuel)
    '€': 1.10,  # Euro (taux de change actuel)
    '₹': 0.014,  # Roupie indienne (taux de change actuel)
    'DEM': 0.62,  # Deutsche Mark (taux de change historique)
    '¥': 0.0093,  # Yen japonais (taux de change actuel)
    'FRF': 0.18,  # Franc français (taux de change historique)
    '₩': 0.00089,  # Won sud-coréen (taux de change actuel)
    'ATS': 0.090,  # Schilling autrichien (taux de change historique)
    'RUR': 0.013,  # Rouble russe (taux de change actuel)
    'DKK': 0.16,  # Couronne danoise (taux de change actuel)
    'CN¥': 0.15,  # Yuan chinois (taux de change actuel)
    'SEK': 0.12,  # Couronne suédoise (taux de change actuel)
    'ITL': 0.00061  # Lire italienne (taux de change historique)
}

# Boucler à travers les valeurs et extraire les symboles et les montants
for i, valeur in enumerate(data_imdb['budget']):
    match = re.match(r'([^$\d]+)(\d+)', valeur)
    if match:
        symbole = match.group(1).strip()
        montant = int(match.group(2))
        if symbole in taux_de_conversion:
            montant *= taux_de_conversion[symbole]
        data_imdb.at[i, 'budget'] = montant

data_imdb['budget'] = data_imdb['budget'].str.replace('$', '')


# Afficher les premières lignes pour vérification
# display(data_imdb)



In [1130]:
display(data_imdb.info())
display(data_allocine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   annee              2245 non-null   int64  
 1   budget             2166 non-null   object 
 2   casting_complet    2245 non-null   object 
 3   casting_principal  2245 non-null   object 
 4   date_sortie_x      0 non-null      float64
 5   director           2245 non-null   object 
 6   duree              2245 non-null   int64  
 7   film_id            2245 non-null   object 
 8   genre              0 non-null      float64
 9   genres             2245 non-null   object 
 10  langue             2244 non-null   object 
 11  nbre_vote          2245 non-null   object 
 12  pays               2245 non-null   object 
 13  pegi               2245 non-null   object 
 14  popularite_score   2245 non-null   object 
 15  scenaristes        2245 non-null   object 
 16  score              2245 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   actors          1680 non-null   object
 1   director        1721 non-null   object
 2   entrees_fr      1726 non-null   int64 
 3   nationalite     1724 non-null   object
 4   semaine_fr      1726 non-null   object
 5   studio          1647 non-null   object
 6   title           1726 non-null   object
 7   titre_original  1699 non-null   object
dtypes: int64(1), object(7)
memory usage: 108.0+ KB


None

In [1131]:
display(data_imdb)

,annee,budget,casting_complet,casting_principal,date_sortie_x,director,duree,film_id,genre,genres,langue,nbre_vote,pays,pegi,popularite_score,scenaristes,score,titre,titre_france,date_sortie_y
0,2011,25000000,"Viola Davis,Emma Stone,Octavia Spencer,Bryce D...","Tate Taylor,Tate Taylor,Kathryn Stockett",NaN,Tate Taylor,146,tt1454029,NaN,Drama,English,492K,"United States,India",PG-13,614,Tate Taylor,8.1,The Help,NaN,02/09/2011
1,1967,3000000,"Dustin Hoffman,Anne Bancroft,Katharine Ross,Wi...","Mike Nichols,Calder Willingham,Buck Henry",NaN,Mike Nichols,106,tt0061722,NaN,"Comedy,Drama,Romance",English,288K,United States,PG,"1,516",Mike Nichols,8.0,The Graduate,NaN,04/09/1968
2,2016,35000000,"Sacha Baron Cohen,Mark Strong,Rebel Wilson,Fre...","Louis Leterrier,Writers,Sacha Baron Cohen",NaN,Louis Leterrier,83,tt3381008,NaN,"Action,Adventure,Comedy","English,Ukrainian",111K,"United Kingdom,United States",R,275,Louis Leterrier,6.2,The Brothers Grimsby,NaN,13/04/2016
3,2023,125000000,"Zachary Levi,Asher Angel,Jack Dylan Grazer,Rac...","David F. Sandberg,Writers,Henry Gayden",NaN,David F. Sandberg,130,tt10151854,NaN,"Action,Adventure,Comedy",English,124K,"United States,Germany,New Zealand,United Kingd...",PG-13,"1,137",David F. Sandberg,5.9,Shazam! Fury of the Gods,NaN,29/03/2023
4,2015,175000000,"Amy Poehler,Bill Hader,Lewis Black,Mindy Kalin...","Pete Docter,Ronnie Del Carmen,Writers",NaN,Pete Docter,95,tt2096673,NaN,"Animation,Adventure,Comedy","English,Portuguese,Latvian,Russian,Ukrainian",783K,United States,PG,638,Pete Docter,8.1,Inside Out,NaN,18/05/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2240,2013,6000000,"Emma Stone,Stephen Merchant,Richard Gere,Liev ...","Directors,Elizabeth Banks,Steven Brill",NaN,Directors,94,tt1333125,NaN,Comedy,English,112K,United States,R,"1,362",Directors,4.3,Movie 43,NaN,19/06/2013
2241,1995,60000000,"Al Pacino,Robert De Niro,Val Kilmer,Jon Voight...","Michael Mann,Michael Mann,Stars",NaN,Michael Mann,170,tt0113277,NaN,"Action,Crime,Drama","English,Spanish",718K,United States,R,263,Michael Mann,8.3,Heat,NaN,21/02/1996
2242,2008,28000000,"Paul Rudd,Seann William Scott,Elizabeth Banks,...","David Wain,Writers,Timothy Dowling",NaN,David Wain,99,tt0430922,NaN,Comedy,English,241K,"Germany,United States",R,"2,234",David Wain,6.8,Role Models,NaN,04/02/2009
2243,2023,200000000,"Anthony Ramos,Dominique Fishback,Luna Lauren V...","Steven Caple Jr.,Writers,Joby Harold",NaN,Steven Caple Jr.,127,tt5090568,NaN,"Action,Adventure,Sci-Fi","English,Spanish",103K,United States,PG-13,624,Steven Caple Jr.,6.0,Transformers: Rise of the Beasts,NaN,07/06/2023
